In [40]:
import pandas as pd
import re 
df = pd.read_csv("/Users/julie/Downloads/SQ25 Pantry Sign-Ups - Mobile Pantry Sign-ups.csv")

                                           Unnamed: 0  \
0                                                 NaN   
1                                                 NaN   
2                                 What Is This Event?   
3   Mobile Pantry Distribution\n\nThis event is si...   
4                                                 NaN   
5                                                 NaN   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9                                                 NaN   
10                                                NaN   
11  \nVolunteering Description:\n\n\nRole: Driver ...   
12                                                NaN   
13                                                NaN   
14                                                NaN   
15                                                NaN   
16                             

go through multiple weeks

In [41]:
import pandas as pd
import re

max_week = 10
shift_dfs = {}

for col in df.columns:
    for idx, cell in df[col].items():
        if pd.notna(cell) and "(Wk" in str(cell):
            day_label = str(cell).strip()

            # Extract the week number
            match = re.search(r"Wk\s*(\d+)", day_label)
            if match:
                week_num = int(match.group(1))
                if week_num > max_week:
                    continue
            else:
                continue

            target_col = col
            start_row = idx

            # Get column index
            try:
                target_col_idx = int(target_col)
            except ValueError:
                target_col_idx = df.columns.get_loc(target_col)

            columns_to_extract = df.iloc[start_row + 1:, target_col_idx:target_col_idx + 4].reset_index(drop=True)

            shift_data = []
            current_time_block = None

            for _, row in columns_to_extract.iterrows():
                if pd.notna(row[0]) and row[0] != "":
                    current_time_block = row[0]
                shift_data.append([day_label, current_time_block, row[1], row[2], row[3]])

            shift_df = pd.DataFrame(shift_data, columns=['Day Label', 'Time Block', 'Role', 'Name', 'Sign-Up'])
            shift_df = shift_df.dropna(subset=['Sign-Up'])
            shift_df = shift_df[(shift_df['Name'].notna()) & (shift_df['Name'] != '')]

            shift_dfs[day_label] = shift_df

# Combine all weekly DataFrames into one
combined_df = pd.concat(shift_dfs.values(), ignore_index=True)

# Extract Week_Label and Date from Day Label
combined_df[['Week_Label', 'Date']] = combined_df['Day Label'].str.extract(r'(\(Wk\s*\d+\s*-\s*\w+\))\s*(.*)')

# Extract week number and location
combined_df['Week'] = combined_df['Week_Label'].str.extract(r'Wk\s*(\d+)').astype(float).astype('Int64')
combined_df['Location'] = combined_df['Week_Label'].str.extract(r'Wk\s*\d+\s*-\s*(\w+)', expand=False)

# Convert Date string to datetime 
current_year = pd.to_datetime("today").year
combined_df['Parsed Date'] = pd.to_datetime(combined_df['Date'] + ' ' + str(current_year), format='%A %m/%d %Y', errors='coerce')

# Reorder columns by dates
combined_df = combined_df[['Week', 'Location', 'Date', 'Parsed Date', 'Time Block', 'Role', 'Name', 'Sign-Up']]

# Sort by Parsed Date in ascending order (earliest first) and reset index
combined_df = combined_df.sort_values(by='Parsed Date', ascending=True).reset_index(drop=True)

# Drop the 'Parsed Date' column
combined_df = combined_df.drop(columns=['Parsed Date'])

combined_df


/var/folders/x_/715mvl2x1h33ww2x2tjg5t5r0000gp/T/ipykernel_3962/3000308738.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notna(row[0]) and row[0] != "":
/var/folders/x_/715mvl2x1h33ww2x2tjg5t5r0000gp/T/ipykernel_3962/3000308738.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_time_block = row[0]
/var/folders/x_/715mvl2x1h33ww2x2tjg5t5r0000gp/T/ipykernel_3962/3000308738.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser

,Week,Location,Date,Time Block,Role,Name,Sign-Up
0,1,SCC,Tuesday 4/1,12PM-1PM,Volunteer 2,Chitra Mukherjee,True
1,1,SCC,Tuesday 4/1,10AM-11AM,Staff Member,Chitra Mukherjee,True
2,1,SCC,Tuesday 4/1,11AM-12PM,Volunteer 1,Chitra Mukherjee,True
3,1,SCC,Tuesday 4/1,12PM-1PM,Volunteer 4,Dominic Olson,True
4,1,SCC,Tuesday 4/1,1PM-2PM,Staff Member,Dominic Olson,True
...,...,...,...,...,...,...,...
63,4,ARC,Thursday 4/24,2PM-3PM,Volunteer 4,Anvi Kalucha,True
64,4,ARC,Thursday 4/24,3PM-4PM,Staff Member,Dominic Olson,True
65,4,ARC,Thursday 4/24,3PM-4PM,Volunteer 1,Sara Molina,True
66,4,ARC,Thursday 4/24,12PM-1PM,Staff Member,Dominic Olson,True
